In [1]:
import os
import spotipy
import pandas as pd
import numpy as np

#Create empty df to store all songs in
df = pd.DataFrame()

#Define filepath of dataset location folder
directory = "C:/Users/trist/OneDrive/Desktop/Dataset"
for file in (os.listdir(directory)):
    if file.endswith(".csv"):
        df_new = pd.read_csv(str(directory+"/"+file))
        df_new = df_new[df_new['target']==1]
        df = df.append(df_new[:1000])
    else:
        continue

In [2]:
df.head()

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target
0,Lucky Man,Montgomery Gentry,spotify:track:4GiXBCUF7H6YfNQsnBRIzl,0.578,0.471,4,-7.270,1,0.0289,0.3680,0.0,0.1590,0.532,133.061,196707,4,30.88059,13,1
1,On The Hotline,Pretty Ricky,spotify:track:1zyqZONW985Cs4osz9wlsu,0.704,0.854,10,-5.477,0,0.1830,0.0185,0.0,0.1480,0.688,92.988,242587,4,41.51106,10,1
4,I Got A Feelin',Billy Currington,spotify:track:1tF370eYXUcWwkIvaq3IGz,0.630,0.764,2,-4.353,1,0.0275,0.3630,0.0,0.1250,0.631,112.098,193760,4,22.62384,10,1
7,Where I Come From,Alan Jackson,spotify:track:6ej1QJ8eIYmhsyTlvgDajy,0.726,0.631,11,-8.136,0,0.0334,0.2200,0.0,0.1930,0.746,124.711,239240,4,35.59732,10,1
8,Nothin' To Die For,Tim McGraw,spotify:track:3lRSz6HujrSy9b3LXg2Kq9,0.481,0.786,10,-5.654,1,0.0288,0.0538,0.0,0.0759,0.389,153.105,253640,4,19.65701,11,1


In [3]:
df_to_feed = df.loc[:,['artist',"track","uri"]]
df_to_feed["uri"].astype('string')
df_to_feed = df_to_feed.reset_index(drop=True)
df_to_feed['genres'] = np.nan

In [4]:
df_to_feed.head()

,artist,track,uri,genres
0,Montgomery Gentry,Lucky Man,spotify:track:4GiXBCUF7H6YfNQsnBRIzl,NaN
1,Pretty Ricky,On The Hotline,spotify:track:1zyqZONW985Cs4osz9wlsu,NaN
2,Billy Currington,I Got A Feelin',spotify:track:1tF370eYXUcWwkIvaq3IGz,NaN
3,Alan Jackson,Where I Come From,spotify:track:6ej1QJ8eIYmhsyTlvgDajy,NaN
4,Tim McGraw,Nothin' To Die For,spotify:track:3lRSz6HujrSy9b3LXg2Kq9,NaN


In [5]:
 #Sign into Spotify API
from spotipy.oauth2 import SpotifyClientCredentials

#Store values for API info
client_id = '265d285ce22e4a3bbbac17b1c94607ae'
client_secret = '285f629d606c4d439e7b55a454a80a70'

#Instantiate Spotipy object via login
client_credentials_manager = SpotifyClientCredentials(client_id,client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
#Create a list to store the genre values to be stored in the df column
genre_contents =[]

#Iterate through each uri in the df
for uri in df['uri']:
    
    #Create empty list for the genres for the song
    song_genres = []
    
    #Get the song metadata 
    track = sp.track(uri)
    
    #Get the artists of the track since Spotify stores genre data at this level 
    track_artists = track['artists']
        
    #Loop through the artists to get the genre data
    for artist in track_artists:
        
        #Get the artist object
        artist_data = sp.artist(artist['id'])
        
        #Add the genres from each artist to the song's raw genre category
        song_genres += artist_data['genres']
    #Compile the song genres into the main list
    genre_contents.append(song_genres)

In [7]:
#Add the genre column 
df_to_feed['genres']=genre_contents

#Drop the URI column since we no longer need it
df_to_export = df_to_feed.drop(columns="uri")

In [8]:
df_to_export.to_csv("Genre_Data.csv",index=True)